# Assignment

### Question 1

Question: Implement a simple Thrift server and client that defines a `Student` struct with fields `name` (string), `age` (integer), and `courses` (list of strings). Include a service `School` with a method `enrollCourse` that takes a `Student` record and a course name, adds the course to the student's course list, and returns the updated `Student` record.

In [1]:
%%writefile ../schema/student.thrift

struct Student {
  1: required string name,
  2: optional i64 age,
  3: optional list<string> courses
}

service School {
    Student enrollCourse(1: required Student student, 2: required string course)
}

Overwriting ../schema/student.thrift


In [2]:
%%writefile ../student_thrift_server.py
import thriftpy2
student_thrift = thriftpy2.load("./schema/student.thrift", module_name="student_thrift")

from thriftpy2.rpc import make_server

class School(object):
    def enrollCourse(self, student, course):
        # Initialize courses list if it's None
        if student.courses is None:
            student.courses = []
        student.courses.append(course)
        return student

server = make_server(student_thrift.School, School(), client_timeout=None)
server.serve()

Overwriting ../student_thrift_server.py


Run `python student_thrift_server.py` in a new terminal to start the server

In [4]:
#import requried thriftpy2, initialise the schema file and make the client
import thriftpy2
student_thrift = thriftpy2.load("../schema/student.thrift", module_name="student_thrift")

from thriftpy2.rpc import make_client

school = make_client(student_thrift.School, timeout=None)

# create student
john = student_thrift.Student(
    name="John", age=21
)
print(john)

# enroll student in course
john = school.enrollCourse(john, "data science")
print (john)


Student(name='John', age=21, courses=None)
Student(name='John', age=21, courses=['data science'])


### Question 2

Question: Implement a simple Protocol Buffers server and client that defines a `Book` message with fields `title` (string), `author` (string), and `page_count` (integer). Include a service `Library` with a method `checkoutBook` that takes a `Book` message and returns the same `Book` message.

In [5]:
%%writefile ../schema/book.proto
syntax = "proto3";

message Book {
  string title = 1;
  string author= 2;
  int64 page_count = 3;
  bool is_available = 4;
}

message CheckoutRequest{
  Book book = 1;
}


service Library {
  rpc checkoutBook(CheckoutRequest) returns (Book) {}
}

Overwriting ../schema/book.proto


Run the following command in a terminal to generate the Python codes:

```bash
python -m grpc_tools.protoc -I./schema --python_out=. --grpc_python_out=. ./schema/book.proto
```

This will generate the following files:

```bash
book_pb2.py
book_pb2_grpc.py
```

In [6]:
%%writefile ../book_protobuf_server.py
from concurrent import futures
import grpc
import book_pb2_grpc


class Library(book_pb2_grpc.LibraryServicer):

  def checkoutBook(self, request, context):
    request.book.is_available = False
    return request.book

server = grpc.server(futures.ThreadPoolExecutor(max_workers=2))
book_pb2_grpc.add_LibraryServicer_to_server(Library(), server)
server.add_insecure_port('[::]:50051')
server.start()
server.wait_for_termination()

Overwriting ../book_protobuf_server.py


Run `python book_protobuf_server.py` in a new terminal to start the server.

In [7]:
import sys
sys.path.append('..')
import grpc
import book_pb2
import book_pb2_grpc


book = book_pb2.Book(title="The Great Gatsby", author="F. Scott Fitzgerald", page_count=180, is_available=True)
print(f"Title: {book.title}")
print(f"Author: {book.author}")
print(f"Page count: {book.page_count}")
print(f"Is available: {book.is_available}")

Title: The Great Gatsby
Author: F. Scott Fitzgerald
Page count: 180
Is available: True


In [8]:
with grpc.insecure_channel('localhost:50051') as channel:
    stub = book_pb2_grpc.LibraryStub(channel)
    updated_book = stub.checkoutBook(book_pb2.CheckoutRequest(book=book))
    print(f"Title: {updated_book.title}")
    print(f"Author: {updated_book.author}")
    print(f"Page count: {updated_book.page_count}")
    print(f"Is available: {updated_book.is_available}")

Title: The Great Gatsby
Author: F. Scott Fitzgerald
Page count: 180
Is available: False
